In [ ]:
import pandas as pd
from prepare_data import load_images_and_annotations
from helpers import plot_image_with_boxes
import numpy as np
from model import define_and_compile_model
from interpret_results import non_max_suppression, get_outputs
import matplotlib.pyplot as plt

In [1]:
!curl -L "https://universe.roboflow.com/ds/ZdIkNMw7q1?key=4rUnbzhtLI" > data/roboflow.zip; unzip data/roboflow.zip; rm data/roboflow.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   890  100   890    0     0    110      0  0:00:08  0:00:08 --:--:--   208
100  178M  100  178M    0     0   545k      0  0:05:34  0:05:34 --:--:--  853k57  0:00:09  0:35:48  783k:55  0:00:24  0:05:31  686k0     0   538k      0  0:05:39  0:00:25  0:05:14  825k  0  0:05:23  0:00:27  0:04:56  891kk      0  0:05:32  0:00:59  0:04:33  580k   593k      0  0:05:08  0:01:06  0:04:02  978k0.4M    0     0   597k      0  0:05:05  0:01:09  0:03:56  825k0   632k      0  0:04:49  0:01:18  0:03:31 1068k0   627k      0  0:04:51  0:01:43  0:03:08  761k7k      0  0:04:51  0:01:45  0:03:06  718k    0  0:04:53  0:01:59  0:02:54  564kk      0  0:05:13  0:02:26  0:02:47  418k   543k      0  0:05:36  0:05:02  0:00:34  413k
Archive:  data/roboflow.zip
 extracting: README.dataset.txt      
 extracting: README.roboflow.txt     
   creating: test/
 ext

In [ ]:
data = pd.read_csv("/data/train/_annotations.csv")
data = data[data["filename"].isin(data['filename'].drop_duplicates()[:3500])]

In [ ]:
input_size = (224, 224)
grid = (7, 7)
images, annotations, num2classes, classes2num, num_classes = load_images_and_annotations(data)
# Print the shape of the loaded data
print('Images shape:', images.shape)
print('Annotations shape:', annotations.shape)

In [ ]:
# Assuming you have already loaded the images and annotations
image_index = np.random.randint(0, images.shape[0])
image = images[image_index]
annotation = annotations[image_index]

# Plot the image with bounding boxes
plot_image_with_boxes(image, annotation, grid, num2classes)

In [ ]:
model = define_and_compile_model(num_classes, grid)
model.summary()

In [ ]:
model.fit(x=images, y=annotations, epochs=250)

In [ ]:
index = np.random.randint(0, images.shape[0])
random_image = images[index]
plt.imshow(random_image)
predicted_boxes = model.predict(random_image.reshape(1, 224, 224, 3))[0]

In [ ]:
score_threshold = 0.5
iou_threshold = 0.5

boxes, scores, pred_classes = get_outputs(predicted_boxes, score_threshold)
selected_boxes, selected_scores, selected_classes = non_max_suppression(boxes, scores, pred_classes, score_threshold, iou_threshold)
plot_image_with_boxes(random_image, selected_boxes, selected_classes)